In [ ]:
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TOKENIZERS_PARALLELISM"] = "true"
load_dotenv()

### Sentence Window Retriever

In [ ]:
from app.rag import RAGBuilder
from app.configs import LLMType, EmbeddingType, IndexType, RetrievalType, IngestionSource, TopicExtractorType
from app.evaluation.EvaluateSummary import complete_evaluator, data_loader

# Create data generator for evaluation
data_generator = data_loader.EvaluationDataGenerator(path="EdinburghNLP/xsum", configuration_name="default")
batch = data_generator.generate(num_samples=3)

# Build query engine
builder = RAGBuilder()
sentence_window_query_engine, build_time = (builder
                .set_llm_type(LLMType.GPT35_TURBO)
                .set_embedding_type(EmbeddingType.BGE_SMALL_EN)
                .set_index_type(IndexType.VECTOR_INDEX)
                .set_ingestion_source_and_path(source=IngestionSource.HF,
                                               paths=[[('EdinburghNLP/xsum', 'default'), ['document']]])
                .set_retrieval_type(RetrievalType.SENTENCE_WINDOW)
                .build()
                )

# Set up evaluator
comp_evaluator = complete_evaluator.CompleteEvaluator(
        query_engine=sentence_window_query_engine,
        app_id="sentence_window",
        eval_batch=batch,
        reset_json=True,
        reset_csv=True
    )
comp_evaluator.evaluate()

### Extractive Summarization

In [ ]:
# Create query engine
builder = RAGBuilder()
extractive_query_engine = (builder
                .set_llm_type(LLMType.GPT35_TURBO)
                .set_retrieval_type(RetrievalType.EXTRACTIVE_RETRIEVAL)
                .build()
                )

# Set up evaluator
comp_evaluator = complete_evaluator.CompleteEvaluator(
        query_engine=extractive_query_engine,
        app_id="extractive",
        eval_batch=batch,
        reset_json=False,
        reset_csv=False
    )
comp_evaluator.evaluate()

### Topic Extraction - LDA

In [ ]:
# Create query engine
builder = RAGBuilder()
lda_query_engine = (builder
                .set_llm_type(LLMType.GPT35_TURBO)
                .set_embedding_type(EmbeddingType.BGE_SMALL_EN)
                .set_index_type(IndexType.VECTOR_INDEX)
                .set_ingestion_source_and_path(source=IngestionSource.HF,
                                               paths=[[('EdinburghNLP/xsum', 'default'), ['document']]])
                .set_retrieval_type(RetrievalType.TOPIC_EXTRACTION)
                .set_topic_extractor_type(TopicExtractorType.LDA)
                .set_debug_mode(True)
                .build()
                )

# Set up evaluator
comp_evaluator = complete_evaluator.CompleteEvaluator(
        query_engine=lda_query_engine,
        app_id="topic_lda",
        eval_batch=batch,
        reset_json=False,
        reset_csv=False
    )
comp_evaluator.evaluate()

### Topic Extraction - BERTopic

In [ ]:
# Create query engine
builder = RAGBuilder()
bertopic_query_engine = (builder
                .set_llm_type(LLMType.GPT35_TURBO)
                .set_embedding_type(EmbeddingType.BGE_SMALL_EN)
                .set_index_type(IndexType.VECTOR_INDEX)
                .set_ingestion_source_and_path(source=IngestionSource.LOCAL,
                                               paths=['data/source-texts']) # As an example, here we use a local pdf.
                .set_retrieval_type(RetrievalType.TOPIC_EXTRACTION)
                .set_topic_extractor_type(TopicExtractorType.BERTOPIC)
                .set_debug_mode(True)
                .build()
                )

# Set up evaluator
comp_evaluator = complete_evaluator.CompleteEvaluator(
        query_engine=bertopic_query_engine,
        app_id="topic_bertopic",
        eval_batch=batch,
        reset_json=False,
        reset_csv=False
    )
comp_evaluator.evaluate()

### Knowledge Graph

The implementation of a knowledge-graph integration is inspired by the following three articles:<br>
[1] [GraphIndex for RAG](https://medium.com/@nebulagraph/graph-rag-the-new-llm-stack-with-knowledge-graphs-e1e902c504ed)<br>
[2] [Concept Graphs for RAG](https://towardsdatascience.com/how-to-convert-any-text-into-a-graph-of-concepts-110844f22a1a)<br>
[3] [LDA for Topic Extraction](https://towardsdatascience.com/document-topic-extraction-with-large-language-models-llm-and-the-latent-dirichlet-allocation-e4697e4dae87)

In [ ]:
# Create query engine
builder = RAGBuilder()
knowledge_graph_query_engine = (builder
                .set_llm_type(LLMType.GPT35_TURBO)
                .set_embedding_type(EmbeddingType.BGE_SMALL_EN)
                .set_index_type(IndexType.KG_INDEX)
                .set_ingestion_source_and_path(source=IngestionSource.HF,
                                               paths=[[('EdinburghNLP/xsum', 'default'), ['document']]])
                .set_retrieval_type(RetrievalType.KG_RETRIEVAL)
                .set_debug_mode(True)
                .build()
                )

# Set up evaluator
comp_evaluator = complete_evaluator.CompleteEvaluator(
        query_engine=knowledge_graph_query_engine,
        app_id="knowledge_graph",
        eval_batch=batch,
        reset_json=False,
        reset_csv=False
    )
comp_evaluator.evaluate()

### No Retrieval

Only use large language model to generate summary. Here to show how to set up large language model using OpenAI or BART

In [ ]:
from openai import OpenAI
from app.llms.bart import BartHuggingFaceLLM
from app.evaluation.EvaluateSummary.prompt_wrapper import with_prompt_template

# Set up BART model
bart = BartHuggingFaceLLM()
@with_prompt_template
def bart_completion(prompt):
    return bart.complete(prompt).text

# Set up GPT model
@with_prompt_template
def gpt_completion(prompt, model="gpt-3.5-turbo", max_tokens=100):
    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a summarization assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.7
    )
    return response.choices[0].message.content